# Your ~~First~~ Second Image Classifier: Using MLP to Classify Images
# Train

The purpose of the original notebook is to correctly classify an image as containing a dog, cat, or panda.
Containing only 3,000 images, the Animals dataset is meant to be another **introductory** dataset
that we can quickly train a KNN model and obtain initial results (no so good accuracy) that has potential to be used as a baseline. 

Let's take the following steps:

1. Encoding target variable
2. Training the MLP model
3. Export the model

<center><img width="600" src="https://drive.google.com/uc?export=view&id=1a-nyAPNPiVh-Xb2Pu2t2p-BhSvHJS0pO"></center>

Now I gonna do the same thing, but using a MLP instead

## Step 01: Setup

Start out by installing the experiment tracking library and setting up your free W&B account:


*   **pip install wandb** – Install the W&B library
*   **import wandb** – Import the wandb library
*   **wandb login** – Login to your W&B account so you can log all your metrics in one place

In [6]:
!pip install wandb -qU

In [7]:
import wandb
wandb.login()

wandb: Currently logged in as: mtxslv (private_project). Use `wandb login --relogin` to force relogin


True

### Import Packages

In [8]:
# import the necessary packages
from imutils import paths
import logging
import os
import cv2
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
#from sklearn.neighbors import KNeighborsClassifier # old stuff
from sklearn.neural_network import MLPClassifier

In [9]:
# configure logging
# reference for a logging obj
logger = logging.getLogger()

# set level of logging
logger.setLevel(logging.INFO)

# create handlers
c_handler = logging.StreamHandler()
c_format = logging.Formatter(fmt="%(asctime)s %(message)s",datefmt='%d-%m-%Y %H:%M:%S')
c_handler.setFormatter(c_format)

# add handler to the logger
logger.handlers[0] = c_handler

## Step 02 Data Segregation

In [10]:
# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
  "project_name": "first_image_classifier",
  "train_feature_artifact": "train_x:latest",
  "train_target_artifact": "train_y:latest",
  #"neighbors": 1, # old stuff
  'hidden_layers':[50,50,50,50],
  "jobs": -1,
  "encoder": "target_encoder",
  "inference_model": "model"
}

## Defining hyperparameters

In [ ]:
mlp_hyperp = {
    'alpha':{
        'values': [0.0001, 0.0005, 0.001, 0.005] # best one 0.0001
    },
    'random_state': {
        'values':[1,5,42] #42 is best
    },
    'architecture':{
        'values':[
            [50,50,50,50],
            [100,50,50,100], # best
            [100,100,100],
            [30,30,30,30,30,30]
        ]
    }
}
sweep_config = {
    'method': 'grid',
    'metric':{
        'name': 'best_loss',
        'goal': 'minimize'
    },
    'parameters': mlp_hyperp
}
sweep_id = wandb.sweep(sweep_config, entity="mtxslv", project=args["project_name"])

Create sweep with ID: igdixpbx
Sweep URL: https://wandb.ai/mtxslv/first_image_classifier/sweeps/igdixpbx


In [ ]:
def train():
  run = wandb.init(entity="mtxslv",project=args["project_name"], job_type="Train")

  #logger.info("Downloading the training data")
  train_x_artifact = run.use_artifact(args["train_feature_artifact"])
  train_x_path = train_x_artifact.file()
  train_y_artifact = run.use_artifact(args["train_target_artifact"])
  train_y_path = train_y_artifact.file()

  # unpacking the artifacts
  train_x = joblib.load(train_x_path)
  train_y = joblib.load(train_y_path)

  configs = {
      'alpha': 0.00001,
      'random_state': 2,
      'architecture': tuple(args['hidden_layers'])
  }
  le = LabelEncoder()
  train_y = le.fit_transform(train_y)
  model = MLPClassifier(hidden_layer_sizes=wandb.config.architecture,
                        #solver=wandb.config.solver,
                        alpha=wandb.config.alpha,
                        #learning_rate=wandb.config.learning_rate,
                        random_state=wandb.config.random_state,
                        max_iter=250)
  # train a k-NN classifier on the raw pixel intensities
  logger.info("[INFO] training MLP classifier...")
  model.fit(train_x, train_y)
  best_loss = model.best_loss_
  wandb.log({'best_loss': best_loss})

In [ ]:
wandb.agent(sweep_id, function=train, count = 48)

wandb: Agent Starting Run: rdjrwk0c with config:
wandb: 	alpha: 0.0001
wandb: 	architecture: [50, 50, 50, 50]
wandb: 	random_state: 1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


16-10-2022 15:33:21 [INFO] training MLP classifier...


best_loss,▁
best_loss,0.40813


wandb: Agent Starting Run: 0aqrwrpe with config:
wandb: 	alpha: 0.0001
wandb: 	architecture: [50, 50, 50, 50]
wandb: 	random_state: 5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


16-10-2022 15:33:46 [INFO] training MLP classifier...


...

best_loss,▁
best_loss,0.62437


wandb: Agent Starting Run: 5p6f3pko with config:
wandb: 	alpha: 0.005
wandb: 	architecture: [30, 30, 30, 30, 30, 30]
wandb: 	random_state: 5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


16-10-2022 15:53:14 [INFO] training MLP classifier...


best_loss,▁
best_loss,0.73218


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ix3afn87 with config:


In [11]:
run = wandb.init(entity="mtxslv",project=args["project_name"], job_type="Train")

#logger.info("Downloading the training data")
train_x_artifact = run.use_artifact(args["train_feature_artifact"])
train_x_path = train_x_artifact.file()
train_y_artifact = run.use_artifact(args["train_target_artifact"])
train_y_path = train_y_artifact.file()

# unpacking the artifacts
train_x = joblib.load(train_x_path)
train_y = joblib.load(train_y_path)

le = LabelEncoder()
train_y = le.fit_transform(train_y)
model = MLPClassifier(hidden_layer_sizes=[100,50,50,100],
                      #solver=wandb.config.solver,
                      alpha = 0.0001,
                      #learning_rate=wandb.config.learning_rate,
                      random_state = 42,
                      max_iter=250)
# train a k-NN classifier on the raw pixel intensities
logger.info("[INFO] training MLP classifier...")
model.fit(train_x, train_y)

wandb: Currently logged in as: mtxslv. Use `wandb login --relogin` to force relogin


16-10-2022 18:21:54 [INFO] training MLP classifier...


MLPClassifier(hidden_layer_sizes=[100, 50, 50, 100], max_iter=250,
              random_state=42)

In [12]:
logger.info("Dumping the model and encoder artifacts to the disk")

# Save the artifacts using joblib
joblib.dump(le, args["encoder"])
joblib.dump(model, args["inference_model"])

16-10-2022 18:22:35 Dumping the model and encoder artifacts to the disk


['model']

In [13]:
# encoder artifact
artifact = wandb.Artifact(args["encoder"],
                          type="INFERENCE_MODEL",
                          description="A json file representing the target encoder"
                          )

logger.info("Logging the target encoder artifact")
artifact.add_file(args["encoder"])
run.log_artifact(artifact)

16-10-2022 18:22:35 Logging the target encoder artifact


In [14]:
# inference model artifact
artifact = wandb.Artifact(args["inference_model"],
                          type="INFERENCE_MODEL",
                          description="A json file representing the inference model"
                          )

logger.info("Logging the inference model artifact")
artifact.add_file(args["inference_model"])
run.log_artifact(artifact)

16-10-2022 18:22:35 Logging the inference model artifact


In [15]:
run.finish()